In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santander-customer-satisfaction/sample_submission.csv
/kaggle/input/santander-customer-satisfaction/train.csv
/kaggle/input/santander-customer-satisfaction/test.csv


In [2]:
dataset = pd.read_csv("/kaggle/input/santander-customer-satisfaction/train.csv")

### 1. Primeiros Passos

Um conjunto de dados com variáveis não-identificáveis, sem colunas não-numéricas. Apesar de seu tamanho, temos +360 colunas de dimensionalidade, além do ID e da Target.

In [3]:
dataset.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


In [5]:
dataset.shape

(76020, 371)

A variável-alvo está severamente desbalanceada. Alguma técnica de oversampling será necessária.

In [6]:
dataset.TARGET.value_counts()

TARGET
0    73012
1     3008
Name: count, dtype: int64

Nenhum valor nulo.

In [7]:
for coluna in dataset.columns:
    numero_faltantes = dataset[coluna].isna().sum()
    if numero_faltantes != 0:
        print(f"A coluna '{coluna}' possui {numero_faltantes} valores nulos.")

#### 1.1 Correlação
Já que todos os valores são numéricos e não existem casos nulos, é possível fazer uma análise de correlação com a variável-alvo.

In [8]:
correlation = dataset.corr()
correlation["TARGET"].sort_values(ascending = False, key = abs)

TARGET                           1.000000
ind_var30                       -0.149811
num_meses_var5_ult3             -0.148253
num_var30                       -0.138289
num_var42                       -0.135693
                                   ...   
num_reemb_var33_hace3                 NaN
num_trasp_var17_out_hace3             NaN
num_trasp_var33_out_hace3             NaN
saldo_var2_ult1                       NaN
saldo_medio_var13_medio_hace3         NaN
Name: TARGET, Length: 371, dtype: float64

Mesmo com 371 colunas, os melhores resultados de correlação não passam dos 15%.

#### 1.2 Busca por Colunas Constantes
Colunas com correlação NaN não variam seus valores. Vamos identificá-las e eliminá-las.

In [9]:
colunas_constantes = [coluna for coluna in dataset.columns if dataset[coluna].nunique() == 1]
print(colunas_constantes)
print(len(colunas_constantes))

['ind_var2_0', 'ind_var2', 'ind_var27_0', 'ind_var28_0', 'ind_var28', 'ind_var27', 'ind_var41', 'ind_var46_0', 'ind_var46', 'num_var27_0', 'num_var28_0', 'num_var28', 'num_var27', 'num_var41', 'num_var46_0', 'num_var46', 'saldo_var28', 'saldo_var27', 'saldo_var41', 'saldo_var46', 'imp_amort_var18_hace3', 'imp_amort_var34_hace3', 'imp_reemb_var13_hace3', 'imp_reemb_var33_hace3', 'imp_trasp_var17_out_hace3', 'imp_trasp_var33_out_hace3', 'num_var2_0_ult1', 'num_var2_ult1', 'num_reemb_var13_hace3', 'num_reemb_var33_hace3', 'num_trasp_var17_out_hace3', 'num_trasp_var33_out_hace3', 'saldo_var2_ult1', 'saldo_medio_var13_medio_hace3']
34


In [10]:
dataset = dataset.drop(columns = colunas_constantes)

In [11]:
dataset.shape

(76020, 337)

In [12]:
correlation = dataset.corr()
correlation["TARGET"].sort_values(ascending = False, key = abs)

TARGET                    1.000000
ind_var30                -0.149811
num_meses_var5_ult3      -0.148253
num_var30                -0.138289
num_var42                -0.135693
                            ...   
saldo_medio_var17_ult3   -0.000328
num_var7_recib_ult1      -0.000290
num_op_var40_ult3        -0.000119
num_var37_med_ult2       -0.000029
imp_ent_var16_ult1       -0.000017
Name: TARGET, Length: 337, dtype: float64

In [13]:
resultado = correlation["TARGET"].sort_values(ascending = False, key = abs)
resultado.head(21)

TARGET                 1.000000
ind_var30             -0.149811
num_meses_var5_ult3   -0.148253
num_var30             -0.138289
num_var42             -0.135693
ind_var5              -0.135349
num_var5              -0.134095
var36                  0.102919
var15                  0.101322
num_var4              -0.080194
num_var35             -0.076872
ind_var8_0             0.046665
num_var8_0             0.046622
ind_var13             -0.039612
ind_var13_0           -0.039471
num_var13             -0.038400
ind_var12_0           -0.038215
num_var13_0           -0.038045
saldo_var30           -0.037092
ind_var39_0           -0.035045
ind_var13_corto       -0.034438
Name: TARGET, dtype: float64

Em uma busca por correlação, essas são as 20 colunas mais relevantes para o conjunto de dados.

In [14]:
COLUNAS_CORRELACAO = set(resultado.head(21).index)

In [15]:
COLUNAS_CORRELACAO

{'TARGET',
 'ind_var12_0',
 'ind_var13',
 'ind_var13_0',
 'ind_var13_corto',
 'ind_var30',
 'ind_var39_0',
 'ind_var5',
 'ind_var8_0',
 'num_meses_var5_ult3',
 'num_var13',
 'num_var13_0',
 'num_var30',
 'num_var35',
 'num_var4',
 'num_var42',
 'num_var5',
 'num_var8_0',
 'saldo_var30',
 'var15',
 'var36'}

### 1.3 Impacto do SMOTE em correlação
A Target está desbalanceada. O SMOTE impacta os valores de correlação?

In [16]:
from imblearn.over_sampling import SMOTE

x = dataset.drop('TARGET', axis=1)
y = dataset["TARGET"]

SEED = 4923

smote = SMOTE(random_state = SEED)
x_smote, y_smote = smote.fit_resample(x, y)

In [17]:
dataset_smote = pd.concat([x_smote, y_smote], axis = 1)

In [18]:
dataset_smote.TARGET.value_counts()

TARGET
0    73012
1    73012
Name: count, dtype: int64

In [19]:
correlation = dataset_smote.corr()
resultado = correlation["TARGET"].sort_values(ascending = False, key = abs)
resultado.head(21)

TARGET                      1.000000
ind_var30                  -0.575943
ind_var5                   -0.555642
num_meses_var5_ult3        -0.487445
num_var30                  -0.449227
num_var42                  -0.441806
num_var5                   -0.432098
num_var4                   -0.407146
var36                       0.304219
num_var35                  -0.300572
var15                       0.256551
ind_var39_0                -0.256537
ind_var41_0                -0.254939
ind_var43_recib_ult1       -0.214124
ind_var12_0                -0.167349
num_meses_var39_vig_ult3   -0.167091
ind_var9_cte_ult1          -0.166339
num_var39_0                -0.161621
ind_var10cte_ult1          -0.161014
num_var41_0                -0.159071
ind_var9_ult1              -0.158546
Name: TARGET, dtype: float64

O uso de SMOTE para balanceamento traz uma perspectiva nova a correlação, com fatores antes irrelevantes agora significativos.

In [20]:
CORRELACAO_COLUNAS_SMOTE_10 = set(resultado.head(11).index)
CORRELACAO_COLUNAS_SMOTE_20 = set(resultado.head(21).index)
CORRELACAO_COLUNAS_SMOTE_30 = set(resultado.head(31).index)
CORRELACAO_COLUNAS_SMOTE_40 = set(resultado.head(41).index)
CORRELACAO_COLUNAS_SMOTE_50 = set(resultado.head(51).index)

### 2. Fazendo uso de Técnicas de Seleção
Além da análise de correlação, vamos fazer uso de algoritmos de técnicas de seleção para encontrar o conjunto de colunas.

#### 2.1 Variance Threshold

A diferença de escala na variância entre colunas no conjunto de dados é imensa. O uso de VarianceThreshold não será efetivo neste cenário, logo, é necessário escalar os dados. Como o StandardScaler iria igualar a variância das colunas, o MinMaxScaler entra em ação.

In [21]:
dataset_smote.var().sort_values(ascending = False)

delta_num_aport_var13_1y3     3.453200e+17
delta_imp_aport_var13_1y3     3.453200e+17
delta_imp_compra_var44_1y3    6.177389e+16
delta_num_compra_var44_1y3    6.177389e+16
delta_num_reemb_var13_1y3     3.225519e+16
                                  ...     
ind_var13_medio_0             1.369628e-05
ind_var13_medio               1.369628e-05
ind_var29                     1.369628e-05
ind_var34_0                   1.369628e-05
ind_var34                     1.369628e-05
Length: 337, dtype: float64

In [22]:
ID = dataset_smote.ID
TARGET = dataset_smote.TARGET

In [23]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()

x_smote = dataset_smote.drop(["ID", "TARGET"], axis = 1)
scaled_x_smote = scaler.fit_transform(x_smote)

scaled_x_smote = pd.DataFrame(scaled_x_smote, columns = x_smote.columns)
scaled_x_smote

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.18,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001545
1,0.999764,0.29,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002003
2,0.999764,0.18,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002822
3,0.999764,0.32,0.000000,0.015130,0.009275,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002671
4,0.999764,0.34,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146019,0.999764,0.41,0.000294,0.007183,0.004403,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008147
146020,0.999764,0.30,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091
146021,0.999764,0.19,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006645
146022,0.999764,0.28,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091


In [24]:
scaled_x_smote.var().sort_values(ascending = False)

ind_var30                    0.248343
ind_var5                     0.240801
var36                        0.208617
num_meses_var5_ult3          0.192129
ind_var41_0                  0.174322
                               ...   
num_reemb_var33_ult1         0.000007
imp_trasp_var33_out_ult1     0.000007
num_trasp_var33_out_ult1     0.000007
delta_imp_aport_var33_1y3    0.000007
delta_num_aport_var33_1y3    0.000007
Length: 335, dtype: float64

In [25]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold = 0.1)

result = selector.fit_transform(scaled_x_smote)
result = pd.DataFrame(result, columns=scaled_x_smote.columns[selector.get_support()])
result

,ind_var5,ind_var30,ind_var39_0,ind_var41_0,var36,num_meses_var5_ult3
0,0.0,0.0,1.0,1.0,1.000000,0.000000
1,0.0,1.0,1.0,1.0,0.030303,0.333333
2,1.0,1.0,1.0,1.0,1.000000,1.000000
3,1.0,1.0,1.0,1.0,0.020202,0.666667
4,0.0,1.0,1.0,1.0,0.010101,1.000000
...,...,...,...,...,...,...
146019,0.0,1.0,1.0,1.0,0.010101,0.666667
146020,0.0,0.0,1.0,1.0,1.000000,0.000000
146021,1.0,1.0,0.0,0.0,0.777778,1.000000
146022,0.0,0.0,1.0,1.0,1.000000,0.000000


In [26]:
selector = VarianceThreshold(threshold = 0.05)

result = selector.fit_transform(scaled_x_smote)
result = pd.DataFrame(result, columns=scaled_x_smote.columns[selector.get_support()])
result

,ind_var5_0,ind_var5,ind_var30,ind_var39_0,ind_var41_0,var36,ind_var10cte_ult1,ind_var9_cte_ult1,ind_var43_recib_ult1,num_meses_var5_ult3,num_meses_var39_vig_ult3
0,1.0,0.0,0.0,1.0,1.0,1.000000,0.0,0.0,0.0,0.000000,0.666667
1,1.0,0.0,1.0,1.0,1.0,0.030303,0.0,0.0,0.0,0.333333,0.666667
2,1.0,1.0,1.0,1.0,1.0,1.000000,0.0,0.0,0.0,1.000000,0.333333
3,1.0,1.0,1.0,1.0,1.0,0.020202,1.0,1.0,0.0,0.666667,0.333333
4,1.0,0.0,1.0,1.0,1.0,0.010101,0.0,0.0,1.0,1.000000,0.666667
...,...,...,...,...,...,...,...,...,...,...,...
146019,0.0,0.0,1.0,1.0,1.0,0.010101,0.0,0.0,0.0,0.666667,0.000000
146020,1.0,0.0,0.0,1.0,1.0,1.000000,0.0,0.0,0.0,0.000000,0.666667
146021,1.0,1.0,1.0,0.0,0.0,0.777778,0.0,0.0,0.0,1.000000,0.000000
146022,0.0,0.0,0.0,1.0,1.0,1.000000,0.0,0.0,0.0,0.000000,0.666667


In [27]:
selector = VarianceThreshold(threshold = 0.01)

result = selector.fit_transform(scaled_x_smote)
result = pd.DataFrame(result, columns=scaled_x_smote.columns[selector.get_support()])
result

,var15,ind_var5_0,ind_var5,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,...,ind_var10_ult1,ind_var10cte_ult1,ind_var9_cte_ult1,ind_var9_ult1,ind_var43_emit_ult1,ind_var43_recib_ult1,num_meses_var5_ult3,num_meses_var12_ult3,num_meses_var13_corto_ult3,num_meses_var39_vig_ult3
0,0.18,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.666667
1,0.29,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.000000,1.0,0.666667
2,0.18,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.333333
3,0.32,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.666667,0.000000,0.0,0.333333
4,0.34,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.000000,0.333333,0.0,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146019,0.41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.000000,0.0,0.000000
146020,0.30,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.666667
146021,0.19,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.000000
146022,0.28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.666667


In [28]:
selector = VarianceThreshold(threshold = 0.001)

result = selector.fit_transform(scaled_x_smote)
result = pd.DataFrame(result, columns=scaled_x_smote.columns[selector.get_support()])
result

,var3,var15,ind_var1_0,ind_var1,ind_var5_0,ind_var5,ind_var8_0,ind_var8,ind_var12_0,ind_var12,...,num_meses_var13_largo_ult3,num_meses_var39_vig_ult3,num_op_var41_efect_ult1,num_op_var39_efect_ult1,num_op_var39_efect_ult3,num_var45_hace2,num_var45_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3
0,0.999764,0.18,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.666667,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.999764,0.29,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.666667,0.0,0.0,0.0,0.000000,0.000000,0.000667,0.000667,0.000535
2,0.999764,0.18,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.333333,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.999764,0.32,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.333333,0.0,0.0,0.0,0.078947,0.059925,0.000000,0.000000,0.000000
4,0.999764,0.34,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.666667,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146019,0.999764,0.41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.002924,0.003745,0.000000,0.000000,0.000000
146020,0.999764,0.30,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.666667,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
146021,0.999764,0.19,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
146022,0.999764,0.28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.666667,0.0,0.0,0.0,0.005848,0.011236,0.000000,0.000000,0.000000


Os resultados mais interessantes gerados pelo algoritmo são os de variância de 1% e 5%. Vamos separá-los.

In [29]:
selector = VarianceThreshold(threshold = 0.05)
result = selector.fit_transform(scaled_x_smote)
result = pd.DataFrame(result, columns=scaled_x_smote.columns[selector.get_support()])

COLUNAS_VARIANCIA_5 = set(result.columns)

selector = VarianceThreshold(threshold = 0.01)
result = selector.fit_transform(scaled_x_smote)
result = pd.DataFrame(result, columns=scaled_x_smote.columns[selector.get_support()])

COLUNAS_VARIANCIA_1 = set(result.columns)

#### 2.2 SelectKBest
O chi2não depende do escalamento do conjunto de dados.

In [30]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [31]:
selector = SelectKBest(chi2, k = 10)
result = selector.fit_transform(scaled_x_smote, TARGET)
result = pd.DataFrame(result, columns=scaled_x_smote.columns[selector.get_support()])

SELECT_10_BEST = set(result.columns)

In [32]:
selector = SelectKBest(chi2, k = 20)
result = selector.fit_transform(scaled_x_smote, TARGET)
result = pd.DataFrame(result, columns=scaled_x_smote.columns[selector.get_support()])

SELECT_20_BEST = set(result.columns)

In [33]:
selector = SelectKBest(chi2, k = 30)
result = selector.fit_transform(scaled_x_smote, TARGET)
result = pd.DataFrame(result, columns=scaled_x_smote.columns[selector.get_support()])

SELECT_30_BEST = set(result.columns)

In [34]:
selector = SelectKBest(chi2, k = 40)
result = selector.fit_transform(scaled_x_smote, TARGET)
result = pd.DataFrame(result, columns=scaled_x_smote.columns[selector.get_support()])

SELECT_40_BEST = set(result.columns)

In [35]:
selector = SelectKBest(chi2, k = 50)
result = selector.fit_transform(scaled_x_smote, TARGET)
result = pd.DataFrame(result, columns=scaled_x_smote.columns[selector.get_support()])

SELECT_50_BEST = set(result.columns)

#### 2.3 Selecionando com Base em Modelos

In [36]:
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [37]:
selector = SelectFromModel(estimator = DecisionTreeClassifier(random_state = SEED))
result = selector.fit_transform(scaled_x_smote, TARGET)
result = pd.DataFrame(result, columns=scaled_x_smote.columns[selector.get_support()])

DECISION_TREE_COLUMNS = set(result.columns)

In [38]:
selector = SelectFromModel(estimator = LogisticRegression(random_state = SEED))
result = selector.fit_transform(scaled_x_smote, TARGET)
result = pd.DataFrame(result, columns=scaled_x_smote.columns[selector.get_support()])

LOGISTIC_REGRESSION_COLUMNS = set(result.columns)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 3. Analisando Resultados
Fazendo uso dos resultados até então, vamos listar as variáveis comuns entre os seletores, e construir um campo conjunto a partir daí.

In [39]:
# Seleções construídas até aqui.
#LOGISTIC_REGRESSION_COLUMNS
#DECISION_TREE_COLUMNS
#SELECT_50_BEST
#SELECT_40_BEST
#SELECT_30_BEST
#SELECT_20_BEST
#SELECT_10_BEST
#COLUNAS_VARIANCIA_5
#COLUNAS_VARIANCIA_1
#CORRELACAO_COLUNAS_SMOTE_10
#CORRELACAO_COLUNAS_SMOTE_20
#CORRELACAO_COLUNAS_SMOTE_30
#CORRELACAO_COLUNAS_SMOTE_40
#CORRELACAO_COLUNAS_SMOTE_50

In [40]:
contador_colunas = {}

# Iterate through each list and update the counts
for i, lista_colunas in enumerate([CORRELACAO_COLUNAS_SMOTE_10, CORRELACAO_COLUNAS_SMOTE_20, CORRELACAO_COLUNAS_SMOTE_30,
                                   CORRELACAO_COLUNAS_SMOTE_40, CORRELACAO_COLUNAS_SMOTE_50, SELECT_10_BEST, SELECT_20_BEST,
                                   SELECT_30_BEST, SELECT_40_BEST, SELECT_50_BEST, COLUNAS_VARIANCIA_1, COLUNAS_VARIANCIA_5,
                                   LOGISTIC_REGRESSION_COLUMNS, DECISION_TREE_COLUMNS]):
    for coluna in lista_colunas:
        if coluna in contador_colunas:
            contador_colunas[coluna].append(i + 1)
        else:
            contador_colunas[coluna] = [i + 1]
            
contador_colunas = dict(sorted(contador_colunas.items(), reverse = True, key = len))

# Print the appearance of columns in each list
for coluna, lista in contador_colunas.items():
    print(f"{coluna}: {len(lista)}")

ind_var30: 13
num_var30: 10
num_meses_var5_ult3: 14
var15: 8
num_var4: 13
ind_var5: 13
num_var35: 9
num_var5: 10
num_var42: 9
TARGET: 5
var36: 13
num_meses_var39_vig_ult3: 8
num_var39_0: 6
ind_var9_cte_ult1: 11
ind_var39_0: 9
num_var41_0: 6
ind_var10cte_ult1: 11
ind_var43_recib_ult1: 12
ind_var9_ult1: 9
ind_var12_0: 11
ind_var41_0: 11
ind_var37_cte: 9
ind_var43_emit_ult1: 9
ind_var12: 9
num_var13_0: 3
ind_var37: 9
ind_var13_0: 8
ind_var13: 8
num_var13: 3
ind_var37_0: 8
ind_var10_ult1: 8
num_var12: 2
num_meses_var12_ult3: 7
saldo_var30: 3
num_meses_var13_corto_ult3: 7
ind_var5_0: 6
ind_var24_0: 8
ind_var24: 6
num_var13_corto_0: 5
ind_var13_corto: 6
ind_var13_corto_0: 6
num_var24: 4
num_aport_var13_hace3: 2
num_var13_corto: 4
num_var30_0: 1
saldo_var13: 1
saldo_var13_corto: 2
ind_var14_0: 5
num_var24_0: 3
saldo_var42: 1
saldo_medio_var13_corto_ult1: 2
ind_var25_cte: 3
ind_var26_cte: 3
ind_var26: 3
ind_var25: 3
ind_var26_0: 3
ind_var25_0: 3
ind_var13_largo: 2
ind_var8: 3
ind_var1_0: 2
ind

In [41]:
del contador_colunas["TARGET"]

In [42]:
resultado_11_aparicoes = [coluna for coluna, lista in contador_colunas.items() if len(lista) >= 11]
resultado_9_aparicoes = [coluna for coluna, lista in contador_colunas.items() if len(lista) >= 9]
resultado_7_aparicoes = [coluna for coluna, lista in contador_colunas.items() if len(lista) >= 7]
resultado_5_aparicoes = [coluna for coluna, lista in contador_colunas.items() if len(lista) >= 5]
resultado_3_aparicoes = [coluna for coluna, lista in contador_colunas.items() if len(lista) >= 3]

In [43]:
len(resultado_11_aparicoes)

10

In [44]:
len(resultado_9_aparicoes)

20

In [45]:
len(resultado_7_aparicoes)

29

In [46]:
len(resultado_5_aparicoes)

37

In [47]:
len(resultado_3_aparicoes)

52

Com esses resultados, vamos filtrar o conjunto de dados original para criar datasets para análise.

In [48]:
scaled_x_smote

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.18,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001545
1,0.999764,0.29,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002003
2,0.999764,0.18,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002822
3,0.999764,0.32,0.000000,0.015130,0.009275,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002671
4,0.999764,0.34,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146019,0.999764,0.41,0.000294,0.007183,0.004403,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008147
146020,0.999764,0.30,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091
146021,0.999764,0.19,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006645
146022,0.999764,0.28,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091


In [49]:
x_selecionado_frequencia_3 = pd.DataFrame({coluna: scaled_x_smote[coluna] for coluna in resultado_3_aparicoes})
x_selecionado_frequencia_5 = pd.DataFrame({coluna: scaled_x_smote[coluna] for coluna in resultado_5_aparicoes})
x_selecionado_frequencia_7 = pd.DataFrame({coluna: scaled_x_smote[coluna] for coluna in resultado_7_aparicoes})
x_selecionado_frequencia_9 = pd.DataFrame({coluna: scaled_x_smote[coluna] for coluna in resultado_9_aparicoes})
x_selecionado_frequencia_11 = pd.DataFrame({coluna: scaled_x_smote[coluna] for coluna in resultado_11_aparicoes})

In [50]:
x_selecionado_frequencia_3.head()

,ind_var30,num_var30,num_meses_var5_ult3,var15,num_var4,ind_var5,num_var35,num_var5,num_var42,var36,...,num_var24_0,ind_var25_cte,ind_var26_cte,ind_var26,ind_var25,ind_var26_0,ind_var25_0,ind_var8,ind_var8_0,num_var8
0,0.0,0.000000,0.000000,0.18,0.000000,0.0,0.000000,0.0,0.000000,1.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.090909,0.333333,0.29,0.142857,0.0,0.083333,0.0,0.000000,0.030303,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.090909,1.000000,0.18,0.142857,1.0,0.083333,0.2,0.166667,1.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.090909,0.666667,0.32,0.428571,1.0,0.250000,0.2,0.166667,0.020202,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.090909,1.000000,0.34,0.142857,0.0,0.083333,0.0,0.166667,0.010101,...,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
x_selecionado_frequencia_5.head()

,ind_var30,num_var30,num_meses_var5_ult3,var15,num_var4,ind_var5,num_var35,num_var5,num_var42,var36,...,ind_var10_ult1,num_meses_var12_ult3,num_meses_var13_corto_ult3,ind_var5_0,ind_var24_0,ind_var24,num_var13_corto_0,ind_var13_corto,ind_var13_corto_0,ind_var14_0
0,0.0,0.000000,0.000000,0.18,0.000000,0.0,0.000000,0.0,0.000000,1.000000,...,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.090909,0.333333,0.29,0.142857,0.0,0.083333,0.0,0.000000,0.030303,...,0.0,0.000000,1.0,1.0,0.0,0.0,0.5,1.0,1.0,0.0
2,1.0,0.090909,1.000000,0.18,0.142857,1.0,0.083333,0.2,0.166667,1.000000,...,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.090909,0.666667,0.32,0.428571,1.0,0.250000,0.2,0.166667,0.020202,...,1.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.090909,1.000000,0.34,0.142857,0.0,0.083333,0.0,0.166667,0.010101,...,0.0,0.333333,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [52]:
x_selecionado_frequencia_7.head()

,ind_var30,num_var30,num_meses_var5_ult3,var15,num_var4,ind_var5,num_var35,num_var5,num_var42,var36,...,ind_var43_emit_ult1,ind_var12,ind_var37,ind_var13_0,ind_var13,ind_var37_0,ind_var10_ult1,num_meses_var12_ult3,num_meses_var13_corto_ult3,ind_var24_0
0,0.0,0.000000,0.000000,0.18,0.000000,0.0,0.000000,0.0,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,1.0,0.090909,0.333333,0.29,0.142857,0.0,0.083333,0.0,0.000000,0.030303,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.000000,1.0,0.0
2,1.0,0.090909,1.000000,0.18,0.142857,1.0,0.083333,0.2,0.166667,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,1.0,0.090909,0.666667,0.32,0.428571,1.0,0.250000,0.2,0.166667,0.020202,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.000000,0.0,0.0
4,1.0,0.090909,1.000000,0.34,0.142857,0.0,0.083333,0.0,0.166667,0.010101,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,1.0


In [53]:
x_selecionado_frequencia_9.head()

,ind_var30,num_var30,num_meses_var5_ult3,num_var4,ind_var5,num_var35,num_var5,num_var42,var36,ind_var9_cte_ult1,ind_var39_0,ind_var10cte_ult1,ind_var43_recib_ult1,ind_var9_ult1,ind_var12_0,ind_var41_0,ind_var37_cte,ind_var43_emit_ult1,ind_var12,ind_var37
0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.090909,0.333333,0.142857,0.0,0.083333,0.0,0.000000,0.030303,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.090909,1.000000,0.142857,1.0,0.083333,0.2,0.166667,1.000000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.090909,0.666667,0.428571,1.0,0.250000,0.2,0.166667,0.020202,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
4,1.0,0.090909,1.000000,0.142857,0.0,0.083333,0.0,0.166667,0.010101,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0


In [54]:
x_selecionado_frequencia_11.head()

,ind_var30,num_meses_var5_ult3,num_var4,ind_var5,var36,ind_var9_cte_ult1,ind_var10cte_ult1,ind_var43_recib_ult1,ind_var12_0,ind_var41_0
0,0.0,0.000000,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,1.0
1,1.0,0.333333,0.142857,0.0,0.030303,0.0,0.0,0.0,0.0,1.0
2,1.0,1.000000,0.142857,1.0,1.000000,0.0,0.0,0.0,0.0,1.0
3,1.0,0.666667,0.428571,1.0,0.020202,1.0,1.0,0.0,0.0,1.0
4,1.0,1.000000,0.142857,0.0,0.010101,0.0,0.0,1.0,1.0,1.0


### 4. Treinamento
Agora, com os conjuntos definidos, é hora de treinar os modelos e avaliar resultados.

In [55]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

linear = SVC(random_state = SEED)
knn = KNeighborsClassifier()
lr = LogisticRegression(random_state = SEED)
dt = DecisionTreeClassifier(random_state = SEED)
rf = RandomForestClassifier(random_state = SEED)
ab = AdaBoostClassifier(random_state = SEED)
gb = GradientBoostingClassifier(random_state = SEED)
extra = ExtraTreesClassifier(random_state = SEED)
mlp = MLPClassifier(random_state = SEED)

In [56]:
from sklearn.model_selection import cross_validate

def avalia_modelo(modelo, x, y, dummy = False, multiclass = False):

    if dummy:
        resultados = cross_validate(modelo, x, y,
                                    scoring = ["accuracy"],
                                    cv = 10, n_jobs = -1)
        print(f'Modelo: {modelo}')


        media_acuracia = sum(resultados["test_accuracy"])/10
        print(f'Acurácia --> {media_acuracia :.2f}')

    elif multiclass:
        resultados = cross_validate(modelo, x, y,
                                    scoring = ["accuracy", "f1_weighted", "precision_weighted", "recall_weighted"],
                                    cv = 10, n_jobs = -1)
        print(f'Modelo: {modelo}')


        media_acuracia = sum(resultados["test_accuracy"])/10
        media_precisao = sum(resultados["test_precision_weighted"])/10
        media_recall = sum(resultados["test_recall_weighted"])/10
        media_f1 = sum(resultados["test_f1_weighted"])/10
        print(f'Acurácia --> {media_acuracia :.2f}')
        print(f'Precisão --> {media_precisao :.2f}')
        print(f'Recall --> {media_recall :.2f}')
        print(f'F1 --> {media_f1 :.2f}')

    else:
        resultados = cross_validate(modelo, x, y,
                                    scoring = ["accuracy", "f1", "precision", "recall"],
                                    cv = 10, n_jobs = -1)
        print(f'Modelo: {modelo}')


        media_acuracia = sum(resultados["test_accuracy"])/10
        media_precisao = sum(resultados["test_precision"])/10
        media_recall = sum(resultados["test_recall"])/10
        media_f1 = sum(resultados["test_f1"])/10
        print(f'Acurácia --> {media_acuracia :.2f}')
        print(f'Precisão --> {media_precisao :.2f}')
        print(f'Recall --> {media_recall :.2f}')
        print(f'F1 --> {media_f1 :.2f}')

#### 4.1 KNN

In [57]:
avalia_modelo(knn, x_selecionado_frequencia_3, TARGET)

Modelo: KNeighborsClassifier()
Acurácia --> 0.83
Precisão --> 0.96
Recall --> 0.69
F1 --> 0.80


In [58]:
avalia_modelo(knn, x_selecionado_frequencia_5, TARGET)

Modelo: KNeighborsClassifier()
Acurácia --> 0.81
Precisão --> 0.97
Recall --> 0.65
F1 --> 0.78


In [59]:
avalia_modelo(knn, x_selecionado_frequencia_7, TARGET)

Modelo: KNeighborsClassifier()
Acurácia --> 0.80
Precisão --> 0.97
Recall --> 0.62
F1 --> 0.75


In [60]:
avalia_modelo(knn, x_selecionado_frequencia_9, TARGET)

Modelo: KNeighborsClassifier()
Acurácia --> 0.75
Precisão --> 0.99
Recall --> 0.51
F1 --> 0.67


In [61]:
avalia_modelo(knn, x_selecionado_frequencia_11, TARGET)

Modelo: KNeighborsClassifier()
Acurácia --> 0.70
Precisão --> 1.00
Recall --> 0.40
F1 --> 0.57


Aparentemente, o resultado da redução de dimensionalidade é uma queda gradual do recall até, eventualmente, comprometer a acurácia do modelo. A precisão é levemente melhorada.

#### 4.2 DecisionTree

In [62]:
avalia_modelo(dt, x_selecionado_frequencia_3, TARGET)

Modelo: DecisionTreeClassifier(random_state=4923)
Acurácia --> 0.90
Precisão --> 0.88
Recall --> 0.93
F1 --> 0.90


In [63]:
avalia_modelo(dt, x_selecionado_frequencia_5, TARGET)

Modelo: DecisionTreeClassifier(random_state=4923)
Acurácia --> 0.90
Precisão --> 0.89
Recall --> 0.91
F1 --> 0.90


In [64]:
avalia_modelo(dt, x_selecionado_frequencia_7, TARGET)

Modelo: DecisionTreeClassifier(random_state=4923)
Acurácia --> 0.90
Precisão --> 0.88
Recall --> 0.91
F1 --> 0.90


In [65]:
avalia_modelo(dt, x_selecionado_frequencia_9, TARGET)

Modelo: DecisionTreeClassifier(random_state=4923)
Acurácia --> 0.84
Precisão --> 0.79
Recall --> 0.93
F1 --> 0.86


In [66]:
avalia_modelo(dt, x_selecionado_frequencia_11, TARGET)

Modelo: DecisionTreeClassifier(random_state=4923)
Acurácia --> 0.82
Precisão --> 0.78
Recall --> 0.90
F1 --> 0.84


#### 4.3 Regressão Logística

In [67]:
avalia_modelo(lr, x_selecionado_frequencia_3, TARGET)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modelo: LogisticRegression(random_state=4923)
Acurácia --> 0.85
Precisão --> 0.84
Recall --> 0.86
F1 --> 0.85


In [68]:
avalia_modelo(lr, x_selecionado_frequencia_5, TARGET)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modelo: LogisticRegression(random_state=4923)
Acurácia --> 0.85
Precisão --> 0.83
Recall --> 0.86
F1 --> 0.85


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [69]:
avalia_modelo(lr, x_selecionado_frequencia_7, TARGET)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modelo: LogisticRegression(random_state=4923)
Acurácia --> 0.85
Precisão --> 0.83
Recall --> 0.87
F1 --> 0.85


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [70]:
avalia_modelo(lr, x_selecionado_frequencia_9, TARGET)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modelo: LogisticRegression(random_state=4923)
Acurácia --> 0.81
Precisão --> 0.77
Recall --> 0.87
F1 --> 0.82


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [71]:
avalia_modelo(lr, x_selecionado_frequencia_11, TARGET)

Modelo: LogisticRegression(random_state=4923)
Acurácia --> 0.79
Precisão --> 0.76
Recall --> 0.85
F1 --> 0.80


#### 4.4 RandomForest

In [72]:
avalia_modelo(rf, x_selecionado_frequencia_3, TARGET)

Modelo: RandomForestClassifier(random_state=4923)
Acurácia --> 0.91
Precisão --> 0.89
Recall --> 0.93
F1 --> 0.91


In [73]:
avalia_modelo(rf, x_selecionado_frequencia_5, TARGET)

Modelo: RandomForestClassifier(random_state=4923)
Acurácia --> 0.90
Precisão --> 0.89
Recall --> 0.91
F1 --> 0.90


In [74]:
avalia_modelo(rf, x_selecionado_frequencia_7, TARGET)

Modelo: RandomForestClassifier(random_state=4923)
Acurácia --> 0.90
Precisão --> 0.89
Recall --> 0.91
F1 --> 0.90


In [75]:
avalia_modelo(rf, x_selecionado_frequencia_9, TARGET)

Modelo: RandomForestClassifier(random_state=4923)
Acurácia --> 0.84
Precisão --> 0.79
Recall --> 0.93
F1 --> 0.86


In [76]:
avalia_modelo(rf, x_selecionado_frequencia_11, TARGET)

Modelo: RandomForestClassifier(random_state=4923)
Acurácia --> 0.82
Precisão --> 0.78
Recall --> 0.90
F1 --> 0.84


Aparentemente, a generalização do RandomForest levou a resultados parecidos em modelos de frequência 3, 5 e 7.

#### 4.5 Ada Boost

In [77]:
avalia_modelo(ab, x_selecionado_frequencia_3, TARGET)

Modelo: AdaBoostClassifier(random_state=4923)
Acurácia --> 0.88
Precisão --> 0.87
Recall --> 0.89
F1 --> 0.88


In [78]:
avalia_modelo(ab, x_selecionado_frequencia_5, TARGET)

Modelo: AdaBoostClassifier(random_state=4923)
Acurácia --> 0.88
Precisão --> 0.87
Recall --> 0.90
F1 --> 0.88


In [79]:
avalia_modelo(ab, x_selecionado_frequencia_7, TARGET)

Modelo: AdaBoostClassifier(random_state=4923)
Acurácia --> 0.88
Precisão --> 0.87
Recall --> 0.89
F1 --> 0.88


In [80]:
avalia_modelo(ab, x_selecionado_frequencia_9, TARGET)

Modelo: AdaBoostClassifier(random_state=4923)
Acurácia --> 0.83
Precisão --> 0.78
Recall --> 0.91
F1 --> 0.84


In [81]:
avalia_modelo(ab, x_selecionado_frequencia_11, TARGET)

Modelo: AdaBoostClassifier(random_state=4923)
Acurácia --> 0.81
Precisão --> 0.77
Recall --> 0.89
F1 --> 0.83


#### 4.6 GradientBoosting

In [82]:
avalia_modelo(gb, x_selecionado_frequencia_3, TARGET)

Modelo: GradientBoostingClassifier(random_state=4923)
Acurácia --> 0.90
Precisão --> 0.89
Recall --> 0.90
F1 --> 0.89


In [83]:
avalia_modelo(gb, x_selecionado_frequencia_5, TARGET)

Modelo: GradientBoostingClassifier(random_state=4923)
Acurácia --> 0.89
Precisão --> 0.89
Recall --> 0.90
F1 --> 0.89


In [84]:
avalia_modelo(gb, x_selecionado_frequencia_7, TARGET)

Modelo: GradientBoostingClassifier(random_state=4923)
Acurácia --> 0.89
Precisão --> 0.89
Recall --> 0.90
F1 --> 0.89


In [85]:
avalia_modelo(gb, x_selecionado_frequencia_9, TARGET)

Modelo: GradientBoostingClassifier(random_state=4923)
Acurácia --> 0.84
Precisão --> 0.79
Recall --> 0.92
F1 --> 0.85


In [86]:
avalia_modelo(gb, x_selecionado_frequencia_11, TARGET)

Modelo: GradientBoostingClassifier(random_state=4923)
Acurácia --> 0.82
Precisão --> 0.77
Recall --> 0.90
F1 --> 0.83


#### 4.8 ExtraTrees

In [87]:
avalia_modelo(extra, x_selecionado_frequencia_3, TARGET)

Modelo: ExtraTreesClassifier(random_state=4923)
Acurácia --> 0.91
Precisão --> 0.89
Recall --> 0.93
F1 --> 0.91


In [88]:
avalia_modelo(extra, x_selecionado_frequencia_5, TARGET)

Modelo: ExtraTreesClassifier(random_state=4923)
Acurácia --> 0.90
Precisão --> 0.89
Recall --> 0.91
F1 --> 0.90


In [89]:
avalia_modelo(extra, x_selecionado_frequencia_7, TARGET)

Modelo: ExtraTreesClassifier(random_state=4923)
Acurácia --> 0.90
Precisão --> 0.89
Recall --> 0.91
F1 --> 0.90


In [90]:
avalia_modelo(extra, x_selecionado_frequencia_9, TARGET)

Modelo: ExtraTreesClassifier(random_state=4923)
Acurácia --> 0.84
Precisão --> 0.79
Recall --> 0.93
F1 --> 0.86


In [91]:
avalia_modelo(extra, x_selecionado_frequencia_11, TARGET)

Modelo: ExtraTreesClassifier(random_state=4923)
Acurácia --> 0.82
Precisão --> 0.78
Recall --> 0.90
F1 --> 0.84


#### 4.9 MLP

In [92]:
avalia_modelo(mlp, x_selecionado_frequencia_3, TARGET)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Modelo: MLPClassifier(random_state=4923)
Acurácia --> 0.90
Precisão --> 0.89
Recall --> 0.91
F1 --> 0.90


In [93]:
avalia_modelo(mlp, x_selecionado_frequencia_5, TARGET)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Modelo: MLPClassifier(random_state=4923)
Acurácia --> 0.90
Precisão --> 0.89
Recall --> 0.90
F1 --> 0.90


In [94]:
avalia_modelo(mlp, x_selecionado_frequencia_7, TARGET)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Modelo: MLPClassifier(random_state=4923)
Acurácia --> 0.89
Precisão --> 0.89
Recall --> 0.90
F1 --> 0.89


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [95]:
avalia_modelo(mlp, x_selecionado_frequencia_9, TARGET)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Modelo: MLPClassifier(random_state=4923)
Acurácia --> 0.83
Precisão --> 0.83
Recall --> 0.85
F1 --> 0.83


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [96]:
avalia_modelo(mlp, x_selecionado_frequencia_11, TARGET)

Modelo: MLPClassifier(random_state=4923)
Acurácia --> 0.82
Precisão --> 0.78
Recall --> 0.89
F1 --> 0.83


#### Resultados
Aparentemente, os algoritmos de melhor performance foram o RandomForest e o ExtraTrees, mas o DecisionTree não perdeu em métricas por muito, e apresenta um melhor potencial de otimização (devido a velocidade de treinamento). E o conjunto com melhor perfomance foi o de maior dimensionalidade, ainda que por pouco.

### 5. Hiperparâmetros

In [97]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [98]:
# DecisionTree
lista_parametros = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'min_samples_split': np.arange(2, 11),
    'min_samples_leaf': np.arange(1, 11)
}

busca = RandomizedSearchCV(DecisionTreeClassifier(random_state = SEED), lista_parametros,
                           cv = 10, n_iter = 30, random_state = SEED,
                           scoring = "accuracy", n_jobs = -1)
busca.fit(x_selecionado_frequencia_3, TARGET)
print(busca.best_estimator_)
print(busca.best_score_ * 100)

DecisionTreeClassifier(criterion='entropy', max_depth=18, min_samples_leaf=3,
                       min_samples_split=8, random_state=4923)
90.29272935354108


In [99]:
avalia_modelo(DecisionTreeClassifier(criterion='entropy', max_depth=18, min_samples_leaf=3,
                                     min_samples_split=8, random_state=4923), 
              x_selecionado_frequencia_3, TARGET)

Modelo: DecisionTreeClassifier(criterion='entropy', max_depth=18, min_samples_leaf=3,
                       min_samples_split=8, random_state=4923)
Acurácia --> 0.90
Precisão --> 0.89
Recall --> 0.92
F1 --> 0.90


A única melhora é um aumento leve na acurácia e na precisão, a custo do recall.

### 6. Conjunto de Teste

In [100]:
alvo = pd.read_csv("/kaggle/input/santander-customer-satisfaction/test.csv")

In [101]:
alvo_id = alvo.ID

In [102]:
alvo = pd.DataFrame({coluna: alvo[coluna] for coluna in x_selecionado_frequencia_3})

In [103]:
alvo.head()

,ind_var30,num_var30,num_meses_var5_ult3,var15,num_var4,ind_var5,num_var35,num_var5,num_var42,var36,...,num_var24_0,ind_var25_cte,ind_var26_cte,ind_var26,ind_var25,ind_var26_0,ind_var25_0,ind_var8,ind_var8_0,num_var8
0,1,3,3,32,1,1,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
1,1,3,3,35,1,1,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
2,1,3,3,23,2,1,6,3,3,3,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,24,0,0,0,0,0,99,...,0,0,0,0,0,0,0,0,0,0
4,1,3,3,23,1,1,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0


In [104]:
scaler = MinMaxScaler()

copia = alvo.copy()
copia = scaler.fit_transform(copia)

alvo = pd.DataFrame(copia, columns = alvo.columns)
alvo.head()

,ind_var30,num_var30,num_meses_var5_ult3,var15,num_var4,ind_var5,num_var35,num_var5,num_var42,var36,...,num_var24_0,ind_var25_cte,ind_var26_cte,ind_var26,ind_var25,ind_var26_0,ind_var25_0,ind_var8,ind_var8_0,num_var8
0,1.0,0.125,1.0,0.27,0.111111,1.0,0.071429,0.25,0.2,0.030303,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.125,1.0,0.30,0.111111,1.0,0.071429,0.25,0.2,0.030303,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.125,1.0,0.18,0.222222,1.0,0.142857,0.25,0.2,0.030303,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000,0.0,0.19,0.000000,0.0,0.000000,0.00,0.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.125,1.0,0.18,0.111111,1.0,0.071429,0.25,0.2,0.030303,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [105]:
model = DecisionTreeClassifier(criterion='entropy', max_depth=18, min_samples_leaf=3,
                                     min_samples_split=8, random_state=4923)

In [106]:
model.fit(x_selecionado_frequencia_3, TARGET)

DecisionTreeClassifier(criterion='entropy', max_depth=18, min_samples_leaf=3,
                       min_samples_split=8, random_state=4923)

In [107]:
resultados = model.predict(alvo)

In [108]:
resultados

array([0, 0, 1, ..., 0, 0, 0])

In [109]:
resposta = pd.concat([alvo_id, pd.Series(resultados, name = "TARGET")], axis = 1)

In [110]:
resposta

,ID,TARGET
0,2,0
1,5,0
2,6,1
3,7,0
4,9,0
...,...,...
75813,151831,0
75814,151832,0
75815,151833,0
75816,151834,0


In [111]:
resposta.to_csv("resposta.csv", index = False)

In [112]:
resposta = resposta.reset_index(drop = True)

In [113]:
resposta

,ID,TARGET
0,2,0
1,5,0
2,6,1
3,7,0
4,9,0
...,...,...
75813,151831,0
75814,151832,0
75815,151833,0
75816,151834,0
